In [1]:
# Standard data analysis libraries
import pandas as pd

# Web scraping libraries
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
import requests
from bs4 import BeautifulSoup
import os

In [2]:
URL_ALL_WATCHES = r"https://www.watchfinder.co.uk/all-watches?orderby=BestMatch&pageno="
URL_BASE = r"https://www.watchfinder.co.uk"

In [3]:
keys = ['id',
        'brand',
        'series',
        'model',
        'price',
        'Year',
        'Box',
        'Papers',
        'Manufacturer warranty',
        'Watchfinder warranty',
        'Availability',
        'Delivery',
        'Returns policy',
        'Product code',
        'Movement',
        'Case size',
        'Case material',
        'Bracelet material',
        'Dial type',
        'Water resistance',
        'Location',
        'img_name']

In [4]:
def g(page_num):
    headers = {'User-Agent': 'Mozilla/5.0'}
    req = Request(URL_ALL_WATCHES + page_num, headers=headers)
    html = urlopen(req)
    bs = BeautifulSoup(html.read(), 'html.parser')
    
    a_tags = bs.find_all('a', {'class': 'prods_name redirect'})
    
    urls = []
    
    for i in range(len(a_tags)):
        urls.append(a_tags[i].attrs['href'])
        
    for i in range(len(urls)):
        urls[i] = urls[i].replace(' ', '%20')
    
    return urls

In [5]:
%%time

urls = []
for i in range(1, 114):
    urls = urls + g(str(i))
    
    print(f'{i+1} / {len(range(1, 114))} completed', end='\r')

Wall time: 4min 23s


In [6]:
for i in range(len(urls)):
    if urls[i].find('é') > 1:
        urls[i] = urls[i].replace('é', '\\xe9')

In [7]:
def f(url, idx):
    headers = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url, headers=headers)
    html = urlopen(req)
    bs = BeautifulSoup(html.read(), 'html.parser')
    
    table_data = bs.find_all('tr')
    
    watches_dict = {}

    for key in keys:
        watches_dict[key] = None
    
    watches_dict['id'] = idx
    
    for i in range(len(table_data)):

        if len(list(table_data[i])) == 5:

            if list(table_data[i])[1].find('div'):
                key = list(list(table_data[i])[1].find('div'))[0]            
                value = list(list(table_data[i])[3])[0]

                watches_dict[key] = value

            else:
                key = list(list(table_data[i])[1])[0]            
                value = list(list(table_data[i])[3])[0]

                watches_dict[key] = value
                
    brand = list(bs.find('span', {'class': 'prod_brand ellipsis'}))[0]
    series = list(bs.find('span', {'class': 'prod_series ellipsis'}))[0]
    model = list(bs.find('span', {'class': 'prod_model ellipsis'}))[0]
    price = bs.find('meta', {'itemprop': 'price'}).attrs['content']
    
    watches_dict['brand'] = brand
    watches_dict['series'] = series
    watches_dict['model'] = model
    watches_dict['price'] = price
    
    try:
        image = bs.find('meta', {'itemprop': 'image'})
        img_url = image.attrs['content']
        img_name = str(idx) + '.jpg'
        watches_dict['img_name'] = img_name
        save_path = os.path.join('./images/', img_name)
        with open(save_path, 'wb') as handle:
            response = requests.get(img_url, stream=True)
            
            if not response.ok:
                print(response)
            
            for block in response.iter_content(1024):
                if not block:
                    break
                    
                handle.write(block)
        
    except:
        pass
        
    return watches_dict

In [15]:
%%time

dicts = []

for idx in range(len(urls)):
    
    try:
        dicts.append(f(URL_BASE + urls[idx], idx))
        
    except Exception as e:
        print(urls[idx])
        print(e)
    
    print(f'{idx+1} / {len(urls)} completed', end='\r')

/Longines/Conquest/L3.777.4.99.6/36600/item/178644?searchId=0aaa4987-c668-4d77-a26e-49a1ef8258f4&rank=19
'NoneType' object has no attribute 'attrs'
/Breguet/Classique/5930BB/12/986/11937/item/183992?searchId=cb545433-ff5d-457f-a372-1f802cdbc666&rank=34
'NoneType' object has no attribute 'attrs'
/Rolex/GMT%20Master%20II/126710%20BLNR/39491/item/182612?searchId=5d1ce02e-13d6-4774-a841-6f14a84718d3&rank=6
'NoneType' object has no attribute 'attrs'
/Rolex/Datejust%2041/126334/34330/item/182658?searchId=9d8ae00a-d82c-4791-abaa-0f1417597c8f&rank=9
'NoneType' object has no attribute 'attrs'
/Rolex/GMT%20Master%20II/126710%20BLRO/35644/item/183549?searchId=9d8ae00a-d82c-4791-abaa-0f1417597c8f&rank=13
'NoneType' object has no attribute 'attrs'
<Response [404]>leted
/Panerai/Luminor%20Base/PAM00112/3772/item/180316?searchId=4fe3e828-2f20-4f39-a9f6-8e83ae7cc46a&rank=2
'NoneType' object has no attribute 'attrs'
/Panerai/Manifattura%20Luminor/PAM00289/36594/item/174017?searchId=d5f6a1b0-fee5-480f-8

In [16]:
big_dict = {}

for k in dicts[0]:
    big_dict[k] = [d[k] for d in dicts]

In [17]:
df = pd.DataFrame.from_dict(big_dict)

In [18]:
df.to_csv('wf_df_raw.csv')

In [ ]:
# df = pd.read_csv('wf_df_raw.csv')